---


**Large MultiLabel Text Classification**
> AM DS1190015
> <br>Apostolos Papatheodorou 

---

In [ ]:
import time
import pickle
import numpy as np
import pandas as pd
from os import listdir
import time; import json
from datetime import date
from os.path import isfile, join

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path to Folder with pickles files (pickles for Raptarchis)

%cd /content/drive/My Drive/dissertation/RAPTARCHIS47k 	
!ls
print()
%cd /content/drive/My Drive/dissertation/thesis 	
!ls


---
# ` Preprocessing Section `  

---



### **Data Loading**

In [ ]:
# Load Training Set from pickles 

with open('pickles/Tr_OnlyFiles.pkl', 'rb') as f:
  tr_onlyfiles=pickle.load(f)

with open('pickles/Tr_Year.pkl', 'rb') as f:
  tr_year=pickle.load(f)

with open('pickles/Tr_Header.pkl', 'rb') as f:
  tr_headers=pickle.load(f)

with open('pickles/Tr_Volume.pkl', 'rb') as f:
  tr_vols=pickle.load(f)

with open('pickles/Tr_Chapter.pkl', 'rb') as f:
  tr_chaptrs=pickle.load(f)

with open('pickles/Tr_Subject.pkl', 'rb') as f:
  tr_subjs=pickle.load(f)

with open('pickles/Tr_Articles.pkl', 'rb') as f:
  tr_artcls=pickle.load(f)


# Load Development Set from pickles 

with open('pickles/Dev_OnlyFiles.pkl', 'rb') as f:
  dev_onlyfiles=pickle.load(f)

with open('pickles/Dev_Year.pkl', 'rb') as f:
  dev_year=pickle.load(f)

with open('pickles/Dev_Volume.pkl', 'rb') as f:
  dev_vols=pickle.load(f)

with open('pickles/Dev_Chapter.pkl', 'rb') as f:
  dev_chaptrs=pickle.load(f)

with open('pickles/Dev_Subject.pkl', 'rb') as f:
  dev_subjs=pickle.load(f)

with open('pickles/Dev_Header.pkl', 'rb') as f:
  dev_headers=pickle.load(f)

with open('pickles/Dev_Articles.pkl', 'rb') as f:
  dev_artcls=pickle.load(f)


# Load Test Set from pickles 

with open('pickles/Test_OnlyFiles.pkl', 'rb') as f:
  test_onlyfiles=pickle.load(f)

with open('pickles/Test_Year.pkl', 'rb') as f:
  test_year=pickle.load(f)

with open('pickles/Test_Volume.pkl', 'rb') as f:
  test_vols=pickle.load(f)

with open('pickles/Test_Chapter.pkl', 'rb') as f:
  test_chaptrs=pickle.load(f)

with open('pickles/Test_Subject.pkl', 'rb') as f:
  test_subjs=pickle.load(f)

with open('pickles/Test_Header.pkl', 'rb') as f:
  test_headers=pickle.load(f)

with open('pickles/Test_Articles.pkl', 'rb') as f:
  test_artcls=pickle.load(f)

In [ ]:
import re
import os
import csv
import nltk
import string
import pandas as pd
import seaborn as sns
nltk.download('stopwords')
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn import preprocessing

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
Gr_STOPWORDS = set(stopwords.words('greek'))
puncList = [".",";",":","!","?","/",",","#","@","$","&",")","(","\"", ">", "<", "=", "{", "}", "|", "-"]
punc="".join(puncList)  #"\\"
t = str.maketrans(dict.fromkeys(punc, " "))

print(string.punctuation)
print(punc)

---
**Input Data:**

* **Text = [ Header + Articles ]**

* **text = [ Header + Articles ] - Stopwords**

---

In [ ]:
# Clear headers and articles columns of Raptarchis DataSet
# Text list removes greeek stopwords but text list keeps them

def Clean_Text(heads, arts):

  Text=[]; text=[]
  for i in range(len(heads)):
    item=heads[i].replace('\n', ' ')+' '+arts[i].replace('\n', ' ')
    item=" ".join(item.split() )
    item=item.lower()
    item=" ".join(item.translate(t).split())
    
    Text.append(item)
  
    tokens = [t for t in item.split(' ') if t not in Gr_STOPWORDS]
    item = " ".join(tokens)
  
    text.append( item.translate(str.maketrans('', '', punc)) )
  return Text, text

tr_Text, tr_text = Clean_Text(tr_headers, tr_artcls)
dev_Text, dev_text = Clean_Text(dev_headers, dev_artcls)
test_Text, test_text = Clean_Text(test_headers, test_artcls)

In [ ]:
print("Dev Set:", len(dev_Text), len(dev_text))
print("Train Set:", len(tr_Text),  len(tr_text))
print("Test Set:", len(test_Text),  len(test_text))

In [ ]:
# Create DataFrame with the Data of Raptarchis DataSet
# Dataframe Columns:[Header, Year, Text/text, Categories: [Volume, Chpater, Subject] ]


# Training Set: 
df = pd.DataFrame({'OnlyFiles':tr_onlyfiles}) # Define the DataFrame 

df["text"]  = tr_text;    df["Text"] =    tr_Text  # [Text, text]
df["Volume"]= tr_vols;    df["Chapter"] = tr_chaptrs; df["Subject"]= tr_subjs
df["Header"]= tr_headers; df["Articles"]= tr_artcls;  df["Year"]   = tr_year


# Testing Set:
df_test = pd.DataFrame({'OnlyFiles':test_onlyfiles})

df_test["text"]  = test_text;    df_test["Text"]    = test_Text   # [Text, text]
df_test["Volume"]= test_vols;    df_test["Chapter"] = test_chaptrs; df_test["Subject"]= test_subjs 
df_test["Header"]= test_headers; df_test["Articles"]= test_artcls;  df_test["Year"]   = test_year

# Validation Set:
df_dev = pd.DataFrame({'OnlyFiles':dev_onlyfiles})

df_dev["text"]  = dev_text;    df_dev["Text"] =    dev_Text  # [Text, text]
df_dev["Volume"]= dev_vols;    df_dev["Chapter"] = dev_chaptrs; df_dev["Subject"]= dev_subjs 
df_dev["Header"]= dev_headers; df_dev["Articles"]= dev_artcls;  df_dev["Year"]   = dev_year

In [ ]:
print(df.count()); print(); print(df_dev.count() ); print(); print(df_test.count() )

In [ ]:
# Keep only Frequent Labels.
# A label is Frequent when has Apprearences > Freq_Num
Freq_Num = 50 

vol_filter = df.groupby('Volume').filter(lambda x: len(x) >= Freq_Num)
ch_filter  = vol_filter.groupby('Chapter').filter(lambda x: len(x) >= Freq_Num)
sub_filter = ch_filter.groupby('Subject').filter(lambda x: len(x) >= Freq_Num )
dffreq= sub_filter

# Few-shot Datasets: [ Chpaters & Subjects ]
few_ch=df.groupby('Chapter').filter(lambda x: len(x) < Freq_Num)
few_sub=df.groupby('Subject').filter(lambda x: len(x) < Freq_Num)

In [ ]:
# Count the number of data 

print(dffreq.count());print() # Frequent Dataset (volumes, chapters subjects)
print(few_ch.count());print() # Few-shot Dataset (chapters)
print(few_sub.count())        # Few-shot Dataset (subjects)

In [ ]:
# Print Unique Classes for Freq-shot Dataset(Vols, Chs, Subs)
print("Freq-shot dataset:  ", dffreq["Volume"].unique().shape, dffreq["Chapter"].unique().shape,
      dffreq["Subject"].unique().shape)

# Print Unique Classes for Few-shot Datasets
print("Few-shot datasets:  ", few_ch["Chapter"].unique().shape, few_sub["Subject"].unique().shape)

In [ ]:
# Dev Set: Keep Values that Already exist in Training Set

df_dev = df_dev[ df_dev['Volume'].isin(  list( dffreq["Volume"].unique()) )]
df_dev = df_dev[ df_dev['Chapter'].isin( list( dffreq["Chapter"].unique()) )]
df_dev = df_dev[ df_dev['Subject'].isin( list( dffreq["Subject"].unique()) )]


# Test Set: Keep Values that Already exist in Training Set

df_test = df_test[ df_test['Volume'].isin(  list( dffreq["Volume"].unique()) )]
df_test = df_test[ df_test['Chapter'].isin( list( dffreq["Chapter"].unique()) )]
df_test = df_test[ df_test['Subject'].isin( list( dffreq["Subject"].unique()) )]

In [ ]:
# Validation Set
print("\t\t==================  VALIDATION SET  ==================\n")

# Print Volume Categories in Train and Dev Set
p1=list( dffreq["Volume"].unique() ); p1.sort(); 
p2=list( df_dev["Volume"].unique() ); p2.sort();
print( len(p1), p1 ); print(  len(p2), p2 ); print()

# Print Chapter Categories in Train and Dev Set
p1=list( dffreq["Chapter"].unique() ); p1.sort(); 
p2=list( df_dev["Chapter"].unique() ); p2.sort()
print( len(p1), p1 ); print(  len(p2), p2 ); print()

# Print Subject Categories in Train and Dev Set
p1=list( dffreq["Subject"].unique() ); p1.sort(); 
p2=list( df_dev["Subject"].unique() ); p2.sort()
print( len(p1), p1 ); print(  len(p2), p2 ); print("\n\n")


# Testing Set
print("\t\t==================  TESTING SET  ==================\n") 

# Print Volume Categories in Train and Test Set
p1=list( dffreq["Volume"].unique() ); p1.sort(); 
p2=list( df_test["Volume"].unique() ); p2.sort();
print( len(p1), p1 ); print(  len(p2), p2 ); print()

# Print Chapter Categories in Train and Test Set
p1=list( dffreq["Chapter"].unique() ); p1.sort(); 
p2=list( df_test["Chapter"].unique() ); p2.sort()
print( len(p1), p1 ); print(  len(p2), p2 ); print()

# Print Subject Categories in Train and Test Set
p1=list( dffreq["Subject"].unique() ); p1.sort(); 
p2=list( df_test["Subject"].unique() ); p2.sort()
print( len(p1), p1 ); print(  len(p2), p2 );

In [ ]:
# For Frequent Dataset

# Count the number of Classes for each hierarchical level
Unique_Vol= len(dffreq.Volume.unique())  # Level-1: Volumes
Unique_Ch = len(dffreq.Chapter.unique()) # Level-2: Chapters
Unique_Sub= len(dffreq.Subject.unique()) # Level-3: Subjects

# Convert Lables to Numerical form
vol_label_dict = {}; ch_label_dict = {}; sub_label_dict = {}


# Convert Volumes-Names to Numbers 
possible_labels = dffreq.Volume.unique()
for index, possible_label in enumerate(possible_labels):
    vol_label_dict[possible_label] = index

# Convert Chapters-Names to Numbers 
possible_labels = dffreq.Chapter.unique()
for index, possible_label in enumerate(possible_labels):
    ch_label_dict[possible_label] = index

# Convert Subjects-Names to Numbers 
possible_labels = dffreq.Subject.unique()
for index, possible_label in enumerate(possible_labels):
    sub_label_dict[possible_label] = index


# Save New Columns to: [Train, Dev, Test] Set

# dffreq dataframe 
dffreq['Vol_label'] = dffreq.Volume.replace(vol_label_dict)
dffreq['Ch_label' ] = dffreq.Chapter.replace(ch_label_dict)
dffreq['Sub_label'] = dffreq.Subject.replace(sub_label_dict)

# df_dev dataframe 
df_dev['Vol_label'] = df_dev.Volume.replace(vol_label_dict)
df_dev['Ch_label' ] = df_dev.Chapter.replace(ch_label_dict)
df_dev['Sub_label'] = df_dev.Subject.replace(sub_label_dict)

# df_test dataframe 
df_test['Vol_label'] = df_test.Volume.replace(vol_label_dict)
df_test['Ch_label' ] = df_test.Chapter.replace(ch_label_dict)
df_test['Sub_label'] = df_test.Subject.replace(sub_label_dict)

In [ ]:
# Print Volumes Chapters and Subjects Numbers 
print(Unique_Vol,  vol_label_dict.items())
print(Unique_Ch,  ch_label_dict.items())
print(Unique_Sub, sub_label_dict.items())

In [ ]:
# Total Number of Test/ Train/ Dev. Instances
print( f"Total Testing Set: {   df_test.OnlyFiles.count()}")
print( f"Total Training Set: {  dffreq.OnlyFiles.count() }")
print( f"Total Developemnt Set: {df_dev.OnlyFiles.count()}")

In [ ]:
# Returns the key of a given Dict Value
def get_key(Dict, val):
    l = []
    for key, value in Dict.items():
         if val == value:
             l.append( key )
    return l

In [ ]:
# Optionally:

# Save Dataframes to pickles
dffreq.to_pickle("pickles/dataframes/TrainDF.pkl")
df_dev.to_pickle("pickles/dataframes/ValidDF.pkl")
df_test.to_pickle("pickles/dataframes/TestDF.pkl")

# Store data (serialize)
with open("pickles/dataframes/VolDict.pkl", 'wb') as handle:
    pickle.dump(vol_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Store data (serialize)
with open("pickles/dataframes/ChDict.pkl", 'wb') as handle:
    pickle.dump(ch_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Store data (serialize)
with open("pickles/dataframes/SubDict.pkl", 'wb') as handle:
    pickle.dump(sub_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

---
**Test DataFrames**

---

In [ ]:
# Checkc Dev Dataframe
df_test.sample(10)

In [ ]:
# Checkc Dev Dataframe
df_dev.sample(10)

In [ ]:
# Check Train Dataframe
dffreq.sample(10)

---
### **Matrices**

---

In [ ]:
import torch

In [ ]:
# Optionally

# Load data from pickles
dffreq_pckl = pd.read_pickle("pickles/dataframes/TrainDF.pkl")
df_dev_pckl = pd.read_pickle("pickles/dataframes/ValidDF.pkl")
df_test_pckl = pd.read_pickle("pickles/dataframes/TestDF.pkl")

# load data (serialize)
with open("pickles/dataframes/VolDict.pkl", 'rb') as handle:
    vol_label_dict_pckl = pickle.load(handle)

# Store data (serialize)
with open("pickles/dataframes/ChDict.pkl", 'rb') as handle:
    ch_label_dict_pckl = pickle.load(handle)

# Store data (serialize)
with open("pickles/dataframes/SubDict.pkl", 'rb') as handle:
    sub_label_dict_pckl = pickle.load(handle)


# Count the number of Classes for each hierarchical level
Unique_Vol= len(dffreq.Volume.unique())  # Level-1: Volumes
Unique_Ch = len(dffreq.Chapter.unique()) # Level-2: Chapters
Unique_Sub= len(dffreq.Subject.unique()) # Level-3: Subjects

In [ ]:
print(dffreq_pckl.equals(dffreq), df_dev_pckl.equals(df_dev), df_test_pckl.equals(df_test), "\n")

print(vol_label_dict)
print(vol_label_dict_pckl)
print((vol_label_dict_pckl==vol_label_dict), "\n" )

print(ch_label_dict)
print(ch_label_dict_pckl)
print((ch_label_dict_pckl==ch_label_dict), "\n" )

print(sub_label_dict)
print(sub_label_dict_pckl)
print((sub_label_dict_pckl==sub_label_dict) )

In [ ]:
dffreq_pckl.head(3)

In [ ]:
dffreq.head(3)

---
**Ch_matrix: The matrix which masks the Chapter predictions.**

---

In [ ]:
# dffreq Columns
print(list(dffreq.columns), '\n'); 

# Print Volumes Chapters and Subjects Numbers 
print(Unique_Vol, vol_label_dict.items())
print(Unique_Ch,  ch_label_dict.items())
print(Unique_Sub, sub_label_dict.items())

In [ ]:
# Ch_matrix: The Matrix which masks the Chapter predictions

# Create np.matrix(( |Volumes|*|Chapters| ))
# Find Unique Volumes 
Vols = dffreq.Volume.unique() 

Vol_2_Ch = {} # Dict (Len = Vol_len) for the chapters of each volume.
Ch_matrix = np.zeros((Unique_Vol, Unique_Ch)) # Matrix: ((|Volumes|*|Chapters|))


for vol in Vols:
  
  l = [];
  i = vol_label_dict[vol]
  UniqChapters = dffreq.loc[dffreq['Volume'] == vol].Chapter.unique()
  print( vol_label_dict.get(vol), vol, UniqChapters )
  
  for ch in UniqChapters:
    j = ch_label_dict.get(ch)
    l.append( j )
    Ch_matrix[i, j] = 1.0

  Vol_2_Ch[i] = l
  print(Vol_2_Ch[i], np.nonzero(Ch_matrix[i]))
  print(Ch_matrix[i,:]); print()

In [ ]:
# Some subjects may be children of more than one Chapter nodes: 
# eg. ["ΔΙΑΦΟΡΕΣ ΔΙΑΤΑΞΕΙΣ", 'ΓΕΝΙΚΕΣ ΔΙΑΤΑΞΕΙΣ' ] =ids [11, 100]

print(get_key(sub_label_dict, 100))

# Subject cat has Chapter parents:
cat = "ΔΙΑΦΟΡΕΣ ΔΙΑΤΑΞΕΙΣ"
c= dffreq[dffreq["Subject"].isin([ cat ]) ]
L =c['Chapter'].unique()
print(len(L), end= ': ')
for l in L:
  print( ch_label_dict[l], l, end=', ' )

In [ ]:
print(Vol_2_Ch)
print(get)

---
**Test Chapter_Matrix**

---

In [ ]:
# Chapter-Matrix Analysis: 
print("Chapter Matrix:", Ch_matrix.shape); print()

print("Sum(Ch_matrix, Columns) len:", len(Ch_matrix.sum(0)),); print(Ch_matrix.sum(0) ); print()

print("Sum(Ch_matrix, Rows) len:", len(Ch_matrix.sum(1)),   ); print(Ch_matrix.sum(1) ) 

In [ ]:
# Test the usage of Ch_matrix
# Suppose that the Vol-predictions are everywhere zeros
# Except for indx1 and idx2 positions.

pred = np.zeros(Unique_Vol)

idx1 = 13; pred[idx1] = 22
idx2 = 20; pred[idx2] = 44

# The dot prodcut of (pred and Ch_matrix) = length of Chapters
npred = np.dot(pred, Ch_matrix)

# Which is zero everwhere apart from the childrens of idx1 and idx2
print(len(Vol_2_Ch[idx1]), len(Vol_2_Ch[idx2]), np.count_nonzero(npred))
print(Vol_2_Ch[idx1], Vol_2_Ch[idx2], np.nonzero(npred))
print(npred)

In [ ]:
# Check the process with pytorch Tensors of Batch_size = 4

# Tensor has dimensions: Batch_size * VOL_Size * CH_Size
s = torch.ones(4, Unique_Vol, Unique_Ch )
s[:,] = torch.from_numpy(Ch_matrix)

# The models output is zeros, except some random positions.
a = torch.zeros(4, Unique_Vol)
a[0,0] = 10; a[1, 2] = 20; a[1, 4] =21; a[2,-1] =100; a[3,5] = 200
a= a.unsqueeze(1)
print(a.shape, a)

# Calculate the batch dot product
ot= torch.bmm(a, s).squeeze(1)
print(ot )

In [ ]:
# Now do the mask of predicted Chapter values
# Supose to be that predictions on Chapter are everywhere ones

# Mask unwanted values
a = torch.ones(4, Unique_Ch)*2
print(a.shape, ot.shape, )

# Make the tensor Masking
ot2 = torch.mul(a, ot)
print(ot2.shape, ot2)

---
**Sub_matrix: The matrix which masks the Subject predictions.**

---

In [ ]:
print(sub_label_dict["ΓΕΝΙΚΕΣ ΔΙΑΤΑΞΕΙΣ"])
print(ch_label_dict); print(sub_label_dict)

In [ ]:
# Sub_matrix: The matrix which masks the Subject predictions

# Create np.matrix(( |Chapters|*|Subjects| ))
# Find Unique Chapters 
UniqChapters = dffreq.Chapter.unique() 

Ch_2_Sub = {} # Dict (Len=Vol_len) for the Subjects of each Chapter.
Sub_matrix = np.zeros((Unique_Ch, Unique_Sub)) # Matrix:((|Volumes|*|Chapters|))

for ch in UniqChapters:
  
  l = [];
  i = ch_label_dict[ch]
  UniqSubjects = dffreq.loc[dffreq['Chapter'] == ch].Subject.unique()
  print( ch_label_dict.get(ch), ch,  UniqSubjects )
  
  for sub in UniqSubjects:
    j = sub_label_dict.get(sub)
    l.append( j )
    Sub_matrix[i, j] = 1

  Ch_2_Sub[i] = l
  print(Ch_2_Sub[i], np.nonzero(Sub_matrix[i]))
  print(Sub_matrix[i,:], "\n");  

---
**Test Subject Matrix**

---

In [ ]:
# Test the usage of Sub_matrix on Batch Matrix Multiplication
# Createe a Tensor (Batch_size, Chapter_Len, Subjects_Len)
s = torch.ones(4, Unique_Ch, Unique_Sub )
s[:,] = torch.from_numpy(Sub_matrix)

# The models output
a = torch.zeros(4, Unique_Ch)
a[0,0] = 10; a[1, 102] = 20; a[1, 79] =21; a[2,-1] =100; a[3,2] = 200
a= a.unsqueeze(1)

ot= torch.bmm(a, s).squeeze(1)

print(a.shape, s.shape)
print(ot )

In [ ]:
# Mask unwanted values example
a = torch.ones(4, Unique_Sub)*2
print(a.shape, ot.shape, )
ot2 = torch.mul(a, ot)
print(ot2.shape, ot2)

---
**Test Subject Matrix**

---

In [ ]:
# print Sub_matrix Analysis:
print("Subject Matrix:", Sub_matrix.shape); print()
print( "Sum(Sub_matrix, Columns) len:", len(Sub_matrix.sum(0)))
print( Sub_matrix.sum(0), ) ; print()
print( "Sum(Sub_matrix, Rows) len:",len(Sub_matrix.sum(1)))
print( Sub_matrix.sum(1), ) 

In [ ]:
# Some subjects may be children of more than one Chapter nodes: 
# eg. ["ΔΙΑΦΟΡΕΣ ΔΙΑΤΑΞΕΙΣ", 'ΓΕΝΙΚΕΣ ΔΙΑΤΑΞΕΙΣ' ] =ids [11, 100]

print(get_key(sub_label_dict, 100))

# Subject cat has Chapter parents:
cat = "ΓΕΝΙΚΕΣ ΔΙΑΤΑΞΕΙΣ"
c= dffreq[dffreq["Subject"].isin([ cat ]) ]
L =c['Chapter'].unique()
print(len(L), end= ': ')
for l in L:
  print( ch_label_dict[l], l, end=', ' )

In [ ]:
# Test the usage of Sub_matrix
# Suppose that the Chapter-predictions are everywhere zeros
# Except for indx1 and idx2 positions.
pred = np.zeros(Unique_Ch)

idx1 = 5; pred[idx1] = 11
idx2 = 6; pred[idx2] = 100

# The dot prodcut of (pred and Ch_matrix) = length of Chapters
npred = np.dot(pred, Sub_matrix)

# Which is zero everwhere apart from the childrens of idx1 and idx2pred = np.zeros(Unique_Ch)
print(npred)
print(len(Ch_2_Sub[idx1]), len(Ch_2_Sub[idx2]), np.count_nonzero(npred))
print(Ch_2_Sub[idx1], Ch_2_Sub[idx2], np.nonzero(npred))

In [ ]:
# Save Ch_matrix and Sub_matrix as pickles
with open('pickles/dataframes/Ch.pkl','wb') as f:
    pickle.dump(Ch_matrix, f)

with open('pickles/dataframes/Sub.pkl','wb') as f:
    pickle.dump(Sub_matrix, f)

---
**END OF PREPROCESSING SECTION**

---

### **DataFrame**

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = mlb.classes_
y_pos = np.arange(len(objects))
performance = datay.sum(axis=0)

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Samples')
plt.title('Category')

plt.show()

In [ ]:
print(df.loc[22].Text)
print(df.loc[22].text)

In [ ]:
df.describe()

In [ ]:
print(df['Volume'].value_counts())

In [ ]:
few_sub.describe()

In [ ]:
few_ch.describe()

In [ ]:
print(few_sub['Chapter'].count())
print(few_ch['Chapter'].value_counts())
#print(few_sub['Subject'].count())
#print(few_sub['Subject'].value_counts())

In [ ]:
print(vol_filter['Volume'].value_counts())
print(ch_filter['Chapter'].value_counts())

In [ ]:
print(df['Chapter'].value_counts())

In [ ]:
print(ch_filter.count()); print('\n')

print(ch_filter['Chapter'].value_counts());  print('\n')

ch_filter.describe()

In [ ]:
print(df['Subject'].value_counts())

In [ ]:
print(sub_filter.count()); print('\n')

print(sub_filter['Subject'].value_counts());  print('\n')

sub_filter.describe()

---

**Data Analysis**
---
---

In [ ]:
df.Header.values

In [ ]:
count_summary = df['Volume'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(count_summary.index, count_summary.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Volume Label', fontsize=12)
plt.show()

count_summary = df['Chapter'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(count_summary.index, count_summary.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Chapter Label', fontsize=12)
plt.show()

count_summary = df['Subject'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(count_summary.index, count_summary.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Subject Class', fontsize=12)
plt.show()

In [ ]:
count_summary = df['Volume'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(count_summary.index, count_summary.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Author Name', fontsize=12)
plt.show()

**Group By Label**

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

wc = WordCloud(background_color="white", max_words=5000, 
               stopwords=STOPWORDS, max_font_size= 50)
# generate word cloud
wc.generate(" ".join(df.Text.values))

# show
plt.imshow(wc, interpolation='bilinear')
plt.title("Words from ALL Author", fontsize=20,color='seagreen')
plt.axis("off")

In [ ]:
wc = WordCloud(background_color="white", max_words=5000, 
               stopwords=Gr_STOPWORDS, max_font_size= 50)
# generate word cloud
wc.generate(" ".join(df.Text.values))

# show
plt.imshow(wc, interpolation='bilinear')
plt.title("Words from ALL Author", fontsize=20,color='seagreen')
plt.axis("off")

In [ ]:
grouped_tags = df.groupby("Volume", sort='count').size().reset_index(name='count')
fig = plt.figure(figsize=(8,6))
grouped_tags.plot(figsize=(8,4), title="Volume frequency")

grouped_tags = df.groupby("Chapter", sort='count').size().reset_index(name='count')
fig = plt.figure(figsize=(8,6))
grouped_tags.plot(figsize=(8,4), title="Chapter frequency")

grouped_tags = df.groupby("Subject", sort='count').size().reset_index(name='count')
fig = plt.figure(figsize=(8,6))
grouped_tags.plot(figsize=(8,4), title="Subject frequency")

grouped_tags = df.groupby("Year", sort='count').size().reset_index(name='count')
fig = plt.figure(figsize=(8,6))
grouped_tags.plot(figsize=(8,4), title="Year frequency")

In [ ]:
counts = df.Volume.value_counts()
firstlast = counts[:5].append(counts[-5:])
firstlast.reset_index(name="count")

In [ ]:
counts = df.Chapter.value_counts()
firstlast = counts[:5].append(counts[-5:])
firstlast.reset_index(name="count")

In [ ]:
counts = df.Subject.value_counts()
firstlast = counts[:5].append(counts[-5:])
firstlast.reset_index(name="count")

In [ ]:
print('Length of Greek Stopwords:', len(Gr_STOPWORDS))
print('Length of French Stopwords:', len(Fr_STOPWORDS))
print('Len. of English Stopwords:', len(En_STOPWORDS))

print()
print(Gr_STOPWORDS)
print(Fr_STOPWORDS)
print(En_STOPWORDS)

---

# **`Training SECTION`**
---

**Install Transformers**


In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
import torch
import itertools
from transformers import *
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Greek-BERT
tokenizer = AutoTokenizer.from_pretrained('nlpaueb/bert-base-greek-uncased-v1')
model = AutoModelWithLMHead.from_pretrained('nlpaueb/bert-base-greek-uncased-v1', output_hidden_states=True)

In [ ]:
# MultiLingual-BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained("bert-base-multilingual-uncased",  output_hidden_states = True,)

In [ ]:
text = "Μια ακόμα ελληνική πρόταση"
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input.keys(), '\n', encoded_input)
print(encoded_input["input_ids"][0].tolist()  )
print(tokenizer.convert_ids_to_tokens(encoded_input["input_ids"][0].tolist()))

print()
output = model(**encoded_input)

print(len(output), output.keys())
print(len(output[0][0]), output[0])
print(len(output[1][0]), output[1][0][0:10])
print()


---
**Start Training**

---

In [ ]:
# Use gpu is available
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(device, device =="cuda" )

In [ ]:
# Parameters Configuration

# Two available Models ["Greek" and "Multilingual"]

BERT_MODEL="Greek"
#BERT_MODEL="Multilingual"

# Defining training variables for training
MAX_LEN = 400
MAX_PATIENCE = 3
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
TEST_BATCH_SIZE  = 4
EPOCHS = 5
LEARNING_RATE = 1e-05


# Define Tokenizers
if (BERT_MODEL=='Greek'):
  print("GREEK-BERT")
  tokenizer = AutoTokenizer.from_pretrained('nlpaueb/bert-base-greek-uncased-v1')

elif (BERT_MODEL=="Multilingual" ):
  print("MultiLingual-BERT")
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# Define a Class for Input Tokenisation
# Functions: Truncate sentences to max_len, padding them, convert tokens to ids

class Triage():
  def __init__(self, dataframe, tokenizer, max_len):
      self.len = len(dataframe)
      self.data = dataframe
      self.tokenizer = tokenizer
      self.max_len = max_len
        
  def __getitem__(self, index):
        
      title = str(self.data.Text[index])
      title = " ".join(title.split())
      inputs = self.tokenizer.encode_plus(
          title, None, 
          add_special_tokens=True,
          max_length=self.max_len,
          pad_to_max_length=True,
          return_token_type_ids=True,
          truncation=True
      )
      ids = inputs['input_ids']
      mask = inputs['attention_mask']

      #print(self.data.OnlyFiles[index], self.data.Vol_label[index], self.data.Ch_label[index], self.data.Sub_label[index], self.data.Text[index])

      return {
          'ids': torch.tensor(ids, dtype=torch.long),
          'mask': torch.tensor(mask, dtype=torch.long),
          'target_vol': torch.tensor(self.data.Vol_label[index], dtype=torch.long),
          'target_ch': torch.tensor(self.data.Ch_label[index], dtype=torch.long),
          'target_sub': torch.tensor(self.data.Sub_label[index], dtype=torch.long),
      } 
    
  def __len__(self):
      return self.len

In [ ]:
# Create the dataset and dataloader for the Model

train_dataset= dffreq.reset_index(drop=True)
dev_dataset  = df_dev.reset_index(drop=True)
test_dataset = df_test.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALID Dataset: {}".format(dev_dataset.shape))
print("TEST Dataset:  {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
develop_set  = Triage(dev_dataset, tokenizer,  MAX_LEN)
testing_set  = Triage(test_dataset, tokenizer, MAX_LEN)

In [ ]:
# convert ids to string
#tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(training_set.__getitem__(3)["ids"]))

In [ ]:
print(training_set.__getitem__(3)["ids"])
print()
print(testing_set.__getitem__(3)["mask"])
print()
print(develop_set.__getitem__(3)["mask"])

In [ ]:
# Passs Data to DataLoader 

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

dev_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
develop_loader  = DataLoader(develop_set, **dev_params)
testing_loader  = DataLoader(testing_set, **test_params)

### Models

In [ ]:
# Define SimpleBERT-Class for training 

class SimpleBERT(torch.nn.Module):
  def __init__(self, bert_model, vol_num, ch_num, sub_num):
      super(SimpleBERT, self).__init__()

      self.vol_num = vol_num  # Number of Volumes  nodes
      self.ch_num = ch_num    # Numver of Chapters nodes
      self.sub_num = sub_num  # Number of Subjects nodes
      self.bert_model=bert_model # String varialble 

      # bert_model: Greek or  Multilingual Edition of BERT
      if (self.bert_model=="Greek"):
        print('GREEK-BERT')
        self.l1 = AutoModelWithLMHead.from_pretrained('nlpaueb/bert-base-greek-uncased-v1', output_hidden_states=True)
      
      elif (self.bert_model=='Multilingual') :
        print('MultiLingual-BERT')
        self.l1 = BertModel.from_pretrained("bert-base-multilingual-uncased",  output_hidden_states = True)


      # Use DNNs after bert (not used now)
      # self.pre_classifier = torch.nn.Linear(768, 768)
      # self.dropout = torch.nn.Dropout(0.3)
      
      # Final Outputs
      self.Vol_Classifier = torch.nn.Linear(768, self.vol_num) # Volume Classifier
      self.Ch_Classifier = torch.nn.Linear(768, self.ch_num)   # Chapter Classifier
      self.Sub_Classifier = torch.nn.Linear(768, self.sub_num) # Subject Classifier

      # Chapter Classifier
      # Subject Classifier
      
      
  def forward(self, input_ids, attention_mask):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      
      
      # Keep Last Layer [CLS] output
      pooler=output_1["hidden_states"][ -1][:, 0] 

      
      # No use of DNNs -- Optionaly --
      #pooler = self.pre_classifier(pooler)
      #pooler = torch.nn.ReLU()(pooler)
      #pooler = self.dropout(pooler)

      # Final Results [Volume, Chapter, Subject]
      vol_output = self.Vol_Classifier(pooler) # Volume Output
      ch_output = self.Ch_Classifier(pooler)   # Chapter Output
      sub_output = self.Sub_Classifier(pooler) # Chapter Output

      return vol_output, ch_output, sub_output

In [ ]:
# Define Last3_BERT-Class for training 

class Last3_BERT(torch.nn.Module):
  def __init__(self, bert_model, vol_num, ch_num, sub_num):
      super(Last3_BERT, self).__init__()

      self.vol_num = vol_num  # Number of Volumes  nodes
      self.ch_num = ch_num    # Numver of Chapters nodes
      self.sub_num = sub_num  # Number of Subjects nodes
      self.bert_model=bert_model # String varialble 

      # bert_model: Greek or  Multilingual Edition of BERT
      if (self.bert_model=="Greek"):
        print('GREEK-BERT')
        self.l1 = AutoModelWithLMHead.from_pretrained('nlpaueb/bert-base-greek-uncased-v1', output_hidden_states=True)
      
      elif (self.bert_model=='Multilingual') :
        print('MultiLingual-BERT')
        self.l1 = BertModel.from_pretrained("bert-base-multilingual-uncased",  output_hidden_states = True)


      # Use DNNs after bert (not used now)
      # self.pre_classifier = torch.nn.Linear(768, 768)
      # self.dropout = torch.nn.Dropout(0.3)
      
      # Final Outputs
      self.Vol_Classifier = torch.nn.Linear(768, self.vol_num) # Volume Classifier
      self.Ch_Classifier = torch.nn.Linear(768, self.ch_num)   # Chapter Classifier
      self.Sub_Classifier = torch.nn.Linear(768, self.sub_num) # Subject Classifier

      # Chapter Classifier
      # Subject Classifier
      
      
  def forward(self, input_ids, attention_mask):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      
      
      # Keep Last Layer [CLS] output
      pooler_1 = output_1["hidden_states"][ -1][:, 0] 
      pooler_2 = output_1["hidden_states"][ -2][:, 0]
      pooler_3 = output_1["hidden_states"][ -3][:, 0]

      
      # No use of DNNs -- Optionaly --
      #pooler = self.pre_classifier(pooler)
      #pooler = torch.nn.ReLU()(pooler)
      #pooler = self.dropout(pooler)

      # Final Results [Volume, Chapter, Subject]
      vol_output = self.Vol_Classifier(pooler_3)  # Volume  Output
      ch_output  = self.Ch_Classifier( pooler_2)  # Chapter Output
      sub_output = self.Sub_Classifier(pooler_1)  # Subject Output

      return vol_output, ch_output, sub_output

In [ ]:
# Define Mask_SimpleBERT-Class for training 

class Mask_SimpleBERT(torch.nn.Module):
  def __init__(self, bert_model, vol_num, ch_num, sub_num, Chapter_matrix, Subject_matrix, Batch_size = 4 ):
      super(Mask_SimpleBERT, self).__init__()

      self.vol_num = vol_num  # Number of Volumes  nodes
      self.ch_num = ch_num    # Numver of Chapters nodes
      self.sub_num = sub_num  # Number of Subjects nodes
      self.bert_model=bert_model # String varialble 

      self.Batch_size = Batch_size  # Batch size    
      self.Chapter_matrix = Chapter_matrix # Matrix_Len = |Volumes|*|Chapters|
      self.Subject_matrix = Subject_matrix # Matrix_Len = |Chapters|*|Subjects|

      # Tensors For Masking
      if (device=="cuda"):
        print("GPU Available")
        self.Chapter_Tensor = torch.ones(self.Batch_size, self.Chapter_matrix.shape[0], self.Chapter_matrix.shape[1] ).to(device)
        self.Subject_Tensor = torch.ones(self.Batch_size, self.Subject_matrix.shape[0], self.Subject_matrix.shape[1] ).to(device) 
      else:
        print("No GPU Available")
        self.Chapter_Tensor = torch.ones(self.Batch_size, self.Chapter_matrix.shape[0], self.Chapter_matrix.shape[1] )
        self.Subject_Tensor = torch.ones(self.Batch_size, self.Subject_matrix.shape[0], self.Subject_matrix.shape[1] ) 
    
      self.Chapter_Tensor[:, ] = torch.from_numpy(self.Chapter_matrix)
      self.Subject_Tensor[:, ] = torch.from_numpy(self.Subject_matrix)


      # bert_model: Greek or  Multilingual Edition of BERT
      if (self.bert_model=="Greek"):
        print('GREEK-BERT')
        self.l1 = AutoModelWithLMHead.from_pretrained('nlpaueb/bert-base-greek-uncased-v1', output_hidden_states=True)
      
      elif (self.bert_model=='Multilingual') :
        print('MultiLingual-BERT')
        self.l1 = BertModel.from_pretrained("bert-base-multilingual-uncased",  output_hidden_states = True)

      
      # Final Outputs
      self.Vol_Classifier = torch.nn.Linear(768, self.vol_num) # Volume Classifier
      self.Ch_Classifier = torch.nn.Linear(768, self.ch_num)   # Chapter Classifier
      self.Sub_Classifier = torch.nn.Linear(768, self.sub_num) # Subject Classifier


      
  def forward(self, input_ids, attention_mask):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      
      # Keep Last Layer [CLS] output
      pooler=output_1["hidden_states"][ -1][:, 0] 

      # Final Results [Volume, Chapter, Subject]
      vol_output = self.Vol_Classifier(pooler) # Volume Output
      ch_output = self.Ch_Classifier(pooler)   # Chapter Output
      sub_output = self.Sub_Classifier(pooler) # Chapter Output


      mch_out  = torch.bmm(vol_output.unsqueeze(1), self.Chapter_Tensor).squeeze(1)
      msub_out = torch.bmm(ch_output.unsqueeze(1), self.Subject_Tensor).squeeze(1)

      # Convert Volume/Chapter outputs to probabilities
      #mch_out = torch.nn.functional.softmax(mch_out)
      #msub_out = torch.nn.functional.softmax(msub_out)

      mch_output  = torch.mul(ch_output, mch_out)
      msub_output = torch.mul(sub_output, msub_out)

      return vol_output, mch_output, msub_output
      #return vol_output, ch_output, sub_output , mch_output, msub_output

In [ ]:
# Define SumLast_BERT-Class for training 

class SumLast_BERT(torch.nn.Module):
  def __init__(self, bert_model, vol_num, ch_num, sub_num):
      super(SumLast_BERT, self).__init__()

      self.vol_num = vol_num  # Number of Volumes  nodes
      self.ch_num = ch_num    # Numver of Chapters nodes
      self.sub_num = sub_num  # Number of Subjects nodes
      self.bert_model=bert_model # String varialble 

      # bert_model: Greek or  Multilingual Edition of BERT
      if (self.bert_model=="Greek"):
        print('GREEK-BERT')
        self.l1 = AutoModelWithLMHead.from_pretrained('nlpaueb/bert-base-greek-uncased-v1', output_hidden_states=True)
      
      elif (self.bert_model=='Multilingual') :
        print('MultiLingual-BERT')
        self.l1 = BertModel.from_pretrained("bert-base-multilingual-uncased",  output_hidden_states = True)


      # Use DNNs after bert (not used now)
      # self.pre_classifier = torch.nn.Linear(768, 768)
      # self.dropout = torch.nn.Dropout(0.3)
      
      # Final Outputs
      self.Vol_Classifier = torch.nn.Linear(768, self.vol_num) # Volume Classifier
      self.Ch_Classifier = torch.nn.Linear(768, self.ch_num)   # Chapter Classifier
      self.Sub_Classifier = torch.nn.Linear(768, self.sub_num) # Subject Classifier

      # Chapter Classifier
      # Subject Classifier
      
      
  def forward(self, input_ids, attention_mask):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      
      
      # Keep Last Layer [CLS] output
      
      pooler = output_1["hidden_states"][-1][:,1:-1].sum(1 ) 
      pooler = pooler/(MAX_LEN-2)
      #print(output_1["hidden_states"][-1][:,1:-1].shape)
      #print(pooler)
      #print(pooler/(MAX_LEN-2))
      #print(pooler.shape)

      
      #pooler=output_1["hidden_states"][ -1][:, 0] 

      
      # No use of DNNs -- Optionaly --
      #pooler = self.pre_classifier(pooler)
      #pooler = torch.nn.ReLU()(pooler)
      #pooler = self.dropout(pooler)

      # Final Results [Volume, Chapter, Subject]
      vol_output = self.Vol_Classifier(pooler) # Volume Output
      ch_output = self.Ch_Classifier(pooler)   # Chapter Output
      sub_output = self.Sub_Classifier(pooler) # Chapter Output

      return vol_output, ch_output, sub_output

In [ ]:
# Define SumLast4_BERT-Class for training 

class SumLast4_BERT(torch.nn.Module):
  def __init__(self, bert_model, vol_num, ch_num, sub_num):
      super(SumLast4_BERT, self).__init__()

      self.vol_num = vol_num  # Number of Volumes  nodes
      self.ch_num = ch_num    # Numver of Chapters nodes
      self.sub_num = sub_num  # Number of Subjects nodes
      self.bert_model=bert_model # String varialble 

      # bert_model: Greek or  Multilingual Edition of BERT
      if (self.bert_model=="Greek"):
        print('GREEK-BERT')
        self.l1 = AutoModelWithLMHead.from_pretrained('nlpaueb/bert-base-greek-uncased-v1', output_hidden_states=True)
      
      elif (self.bert_model=='Multilingual') :
        print('MultiLingual-BERT')
        self.l1 = BertModel.from_pretrained("bert-base-multilingual-uncased",  output_hidden_states = True)


      # Use DNNs after bert (not used now)
      # self.pre_classifier = torch.nn.Linear(768, 768)
      # self.dropout = torch.nn.Dropout(0.3)
      
      # Final Outputs
      self.Vol_Classifier = torch.nn.Linear(768, self.vol_num) # Volume Classifier
      self.Ch_Classifier = torch.nn.Linear(768, self.ch_num)   # Chapter Classifier
      self.Sub_Classifier = torch.nn.Linear(768, self.sub_num) # Subject Classifier

      # Chapter Classifier
      # Subject Classifier
      
      
  def forward(self, input_ids, attention_mask):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      
      
      # Keep Last Layer [CLS] output
      
      pooler_1 = output_1["hidden_states"][-1][:,1:-1].sum(1 ) 
      pooler_1 = pooler_1/(MAX_LEN-2)

      pooler_2 = output_1["hidden_states"][-2][:,1:-1].sum(1 ) 
      pooler_2 = pooler_2/(MAX_LEN-2)

      pooler_3 = output_1["hidden_states"][-3][:,1:-1].sum(1 ) 
      pooler_3 = pooler_3/(MAX_LEN-2)

      pooler_4 = output_1["hidden_states"][-4][:,1:-1].sum(1 ) 
      pooler_4 = pooler_4/(MAX_LEN-2)

      pooler = (pooler_1 + pooler_2 + pooler_2 + pooler_4)/4

      # Final Results [Volume, Chapter, Subject]
      vol_output = self.Vol_Classifier(pooler) # Volume Output
      ch_output = self.Ch_Classifier(pooler)   # Chapter Output
      sub_output = self.Sub_Classifier(pooler) # Chapter Output

      return vol_output, ch_output, sub_output 

### **Training**

In [ ]:
# Define an instance of the BERT model

# All the Available Bert-Models

#model = SimpleBERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub,)
#model = Last3_BERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub,)
#model = Mask_SimpleBERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub,
#                        Chapter_matrix= Ch_matrix, Subject_matrix=Sub_matrix)
model = SumLast4_BERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub,)


model.to(device)

# Define the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
"""
for i ,data in enumerate(develop_loader, 0):
  ids = data['ids'].to(device, dtype = torch.long)
  mask = data['mask'].to(device, dtype = torch.long)

  # Target values for Volume, Chapter and Subject Levels
  target_vol = data['target_vol'].to(device, dtype = torch.long)
  target_ch = data['target_ch'].to(device, dtype = torch.long)
  target_sub = data['target_sub'].to(device, dtype = torch.long)

  vol, ch, sub  = model(ids, mask)  

  
  print(Ch_matrix.shape, type(Ch_matrix))
  print(Sub_matrix.shape, type(Sub_matrix))
  print(vol[0].shape, type(target_sub))
  print()


  vol_loss = loss_function(vol, target_vol)#.item()
  ch_loss  = loss_function(ch , target_ch ).item()
  sub_loss = loss_function(sub, target_sub).item()
  print(vol_loss, ch_loss, sub_loss)

  vol_big_val, vol_big_idx = torch.max(vol.data, dim=1)
  print(vol_big_idx, target_vol)


  ch_big_val, ch_big_idx  = torch.max(ch.data, dim=1)
  print(ch_big_idx, target_ch)
  #mch_big_val, mch_big_idx  = torch.max(mch.data, dim=1)
  #print("m", mch_big_idx, target_ch, "\n")

  sub_big_val, sub_big_idx = torch.max(sub.data, dim=1)
  print(sub_big_idx, target_sub )
  #msub_big_val, msub_big_idx = torch.max(msub.data, dim=1)
  #print("m", msub_big_idx, target_sub, "\n" )


  vol_acc, inst1 = calculate_accu(vol_big_idx, target_vol) 
  ch_acc,  inst2 = calculate_accu(ch_big_idx, target_ch)   
  sub_acc, inst3 = calculate_accu(sub_big_idx, target_sub) 

  insts= torch.tensor([ inst1.tolist(), inst2.tolist(), inst3.tolist()])
  print(insts)

  p1=torch.ones(4)*3
  total_acc = (insts.sum(dim=0)==p1).sum().item()/4
  print(f"Acc: {total_acc} [ {vol_acc}, {ch_acc}, {sub_acc}]" )


  break
"""

In [ ]:
# Function to calcuate the Class accuracy 
def calculate_accu(big_idx, targets):
  
  n_correct = (big_idx==targets).sum().item()
  return n_correct/targets.shape[0]  , big_idx==targets

In [ ]:
# Training Function tuning the GreekBERT model

def train(epoch, check_point=150, BATCH_SIZE=TRAIN_BATCH_SIZE ):

    tr_loss=[];     tr_acc=[]  # Loss/Acc-list for each iteration
    tr_vol_acc=[];  tr_ch_acc=[];  tr_sub_acc=[]; # Save acc for each level 
    tr_vol_loss=[]; tr_ch_loss=[]; tr_sub_loss=[];# Save Loss for each level 

    model.train()
    for i ,data in enumerate(training_loader, 0):    
        ids = data['ids'].to(device, dtype = torch.long)
        mask= data['mask'].to(device, dtype = torch.long)

        # Target values for Volume, Chapter and Subject Levels
        target_vol = data['target_vol'].to(device, dtype = torch.long)
        target_ch  = data['target_ch'].to(device, dtype = torch.long )
        target_sub = data['target_sub'].to(device, dtype = torch.long)
        if (target_vol.shape[0]!=BATCH_SIZE): break


        vol, ch, sub = model(ids, mask) # Model prediction
        

        # Loss estimation (sum losses of each category)
        vol_loss = loss_function(vol, target_vol)#.item()
        ch_loss  = loss_function(ch , target_ch )#.item()
        sub_loss = loss_function(sub, target_sub)#.item()

        loss = vol_loss + ch_loss + sub_loss
        

        # Calculate accuracy       
        vol_big_val, vol_big_idx = torch.max(vol.data, dim=1)
        ch_big_val, ch_big_idx  = torch.max(ch.data, dim=1)
        sub_big_val, sub_big_idx = torch.max(sub.data, dim=1)
      
        vol_acc, inst1 = calculate_accu(vol_big_idx, target_vol) # Acc. For Volumes
        ch_acc,  inst2 = calculate_accu(ch_big_idx, target_ch)   # Acc. For Chapters
        sub_acc, inst3 = calculate_accu(sub_big_idx, target_sub) # Acc. For Subjects

        insts= torch.tensor([ inst1.tolist(), inst2.tolist(), inst3.tolist()])

        total_acc = (insts.sum(dim=0)==p1).sum().item()/4  # Acc for [Volumes && Chapaers && Subjects]
              

        # Save Loss and Accuracy for the Current Iteration
        tr_vol_loss.append(vol_loss.item()); tr_vol_acc.append(vol_acc);
        tr_ch_loss.append(ch_loss.item());   tr_ch_acc.append(ch_acc);
        tr_sub_loss.append(sub_loss.item()); tr_sub_acc.append(sub_acc);
        tr_loss.append(loss.item());         tr_acc.append(total_acc);
    
        # Upadate Weights
        optimizer.zero_grad(); loss.backward(); optimizer.step()

        # Check point to Print outputs during the training process
        if ( (i % check_point == 0) and (i!=0) ):
          
          print(f"\n\n\t **************************    Train--Check--point {i}      **************************")
          loss_step = sum(tr_loss[-check_point:]) / len( tr_loss[-check_point:] )
          
          loss_vol_step = sum(tr_vol_loss[-check_point:]) / len( tr_vol_loss[-check_point:] )
          loss_ch_step  = sum(tr_ch_loss[-check_point: ]) / len( tr_ch_loss[-check_point: ] )
          loss_sub_step = sum(tr_sub_loss[-check_point:]) / len( tr_sub_loss[-check_point:] )
          
          
          acc_step  = sum(tr_acc[-check_point:]) / len(tr_acc[-check_point:])

          acc_vol_step = sum(tr_vol_acc[-check_point:]) / len(tr_vol_acc[-check_point:])
          acc_ch_step  = sum(tr_ch_acc[-check_point: ]) / len(tr_ch_acc[-check_point: ])
          acc_sub_step = sum(tr_sub_acc[-check_point:]) / len(tr_sub_acc[-check_point:])

          print(f"\t Training Acc. per {check_point} last steps: {acc_step} [ {acc_vol_step}, {acc_ch_step}, {acc_sub_step}] ")
          print(f"\t Training Loss per {check_point} last steps: {loss_step} [ {loss_vol_step}, {loss_ch_step}, {loss_sub_step}] ")
          
          print(f"\n\t Last {15} predictions:")
          print('\t', [np.round(ls,3) for ls in tr_loss[-10:] ])
          print('\t', tr_acc[-15:] )
          print()

          # Print --Optionally--
          print("\n\tLast Instance:")
          print("\t\t",vol_big_idx, target_vol);  print('\t\t', ch_big_idx, target_ch); 
          print("\t\t",sub_big_idx, target_sub);  print('\n\t\t', insts)
          print()
          print(f"\t\tAcc: {total_acc} [ {vol_acc}, {ch_acc}, {sub_acc}]")
          print(f"\t\tLoss: {loss} [ {vol_loss}, {ch_loss}, {sub_loss}]" )
          print(f"\n\n")

        #if (i==200):break
        if (i%10==0):print( i, end=" " )
          
    # Epoch Loss & Accuracy
    ep_loss = sum(tr_loss)/len(tr_loss);  ep_acc = sum(tr_acc)/len(tr_acc);

    ep_vol_loss = sum(tr_vol_loss)/len(tr_vol_loss); ep_vol_acc = sum(tr_vol_acc)/len(tr_vol_acc);  
    ep_ch_loss  = sum(tr_ch_loss )/len(tr_ch_loss ); ep_ch_acc  = sum(tr_ch_acc )/len(tr_ch_acc );
    ep_sub_loss = sum(tr_sub_loss)/len(tr_sub_loss); ep_sub_acc = sum(tr_sub_acc)/len(tr_sub_acc);

    print("\n")
    print(f"End of Training Epoch {epoch}")
    print(f"Epoch {epoch} Loss: {ep_loss} [ {ep_vol_loss}, {ep_ch_loss}, {ep_sub_loss}] ")
    print(f"Epoch {epoch} Accr: {ep_acc } [ {ep_vol_acc }, {ep_ch_acc }, {ep_sub_acc }]\n")    
    
    print(f"Training Loss Epoch: {[ np.round(rloss , 3) for rloss in tr_loss]}")
    print(f"Training Accuracy Epoch: {tr_acc}")

    return ep_loss, ep_vol_loss, ep_ch_loss, ep_sub_loss, ep_acc, ep_vol_acc, ep_ch_acc, ep_sub_acc

In [ ]:
# Similar to train-method define valid funtion is used for testing and validation
def valid(ModeL, tst_loader, check_point=150, BATCH_SIZE = VALID_BATCH_SIZE, MODE="DEV" ):

    dev_loss=[];     dev_acc=[]  # Loss/Acc-list for each iteration
    dev_vol_acc=[];  dev_ch_acc=[];  dev_sub_acc=[]; # Save acc for each level 
    dev_vol_loss=[]; dev_ch_loss=[]; dev_sub_loss=[];# Save Loss for each level 

    ModeL.eval()
    
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
      
      for i ,data in enumerate(tst_loader, 0):
                
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
    
            # Target values for Volume, Chapter and Subject Levels
            target_vol = data['target_vol'].to(device, dtype = torch.long)
            target_ch = data['target_ch'].to(device, dtype = torch.long)
            target_sub = data['target_sub'].to(device, dtype = torch.long) 
            if (target_vol.shape[0]!=BATCH_SIZE): break

            vol, ch, sub = ModeL(ids, mask) # ModeL prediction
            
            # Loss estimation (sum losses of each category)
            vol_loss = loss_function(vol, target_vol)#.item()
            ch_loss  = loss_function(ch , target_ch )#.item()
            sub_loss = loss_function(sub, target_sub)#.item()
    
            loss = vol_loss + ch_loss + sub_loss    


            # Calculate accuracy 
            vol_big_val, vol_big_idx = torch.max(vol.data, dim=1)
            ch_big_val, ch_big_idx  = torch.max(ch.data, dim=1)
            sub_big_val, sub_big_idx = torch.max(sub.data, dim=1)
          
            vol_acc, inst1 = calculate_accu(vol_big_idx, target_vol) # Acc. For Volumes
            ch_acc,  inst2 = calculate_accu(ch_big_idx, target_ch)   # Acc. For Chapters
            sub_acc, inst3 = calculate_accu(sub_big_idx, target_sub) # Acc. For Subjects
    
            insts= torch.tensor([ inst1.tolist(), inst2.tolist(), inst3.tolist()])
    
            total_acc = (insts.sum(dim=0)==p1).sum().item()/4  # Acc for [Volumes && Chapaers && Subjects]

            # Save Loss and Accuracy for the Current Iteration
            dev_vol_loss.append(vol_loss.item()); dev_vol_acc.append(vol_acc);
            dev_ch_loss.append(ch_loss.item());   dev_ch_acc.append(ch_acc);
            dev_sub_loss.append(sub_loss.item()); dev_sub_acc.append(sub_acc);
            dev_loss.append(loss.item());         dev_acc.append(total_acc);

            if ( (i % check_point == 0) and (i!=0) ):

              print(f"\n\n\t **************************    {MODE}--Check--point {i}      **************************")
              loss_step = sum(dev_loss[-check_point:]) / len( dev_loss[-check_point:] )
              
              loss_vol_step = sum(dev_vol_loss[-check_point:]) / len( dev_vol_loss[-check_point:] )
              loss_ch_step  = sum(dev_ch_loss[-check_point: ]) / len( dev_ch_loss[-check_point: ] )
              loss_sub_step = sum(dev_sub_loss[-check_point:]) / len( dev_sub_loss[-check_point:] )
              
              
              acc_step  = sum(dev_acc[-check_point:]) / len(dev_acc[-check_point:])
    
              acc_vol_step = sum(dev_vol_acc[-check_point:]) / len(dev_vol_acc[-check_point:])
              acc_ch_step  = sum(dev_ch_acc[-check_point: ]) / len(dev_ch_acc[-check_point: ])
              acc_sub_step = sum(dev_sub_acc[-check_point:]) / len(dev_sub_acc[-check_point:])
    
              print(f"\t Dev Acc. per {check_point} last steps: {acc_step} [ {acc_vol_step}, {acc_ch_step}, {acc_sub_step}] ")
              print(f"\t Dev Loss per {check_point} last steps: {loss_step} [ {loss_vol_step}, {loss_ch_step}, {loss_sub_step}]\n")
              print(f"\t Last {15} values")
              print('\t', dev_loss[-15:])
              print('\t', dev_acc[-15:] )
              print()
    
              # Print --Optionally--
              print("\tLast Instance:")
              print("\t\t",vol_big_idx, target_vol);  print("\t\t",ch_big_idx, target_ch); 
              print("\t\t",sub_big_idx, target_sub);  print("\t\t",insts)
              print()
              print(f"\t\tAcc: {total_acc} [ {vol_acc}, {ch_acc}, {sub_acc}]")
              print(f"\t\tLoss: {loss} [ {vol_loss}, {ch_loss}, {sub_loss}]" )
              print('\n')

            if (i%15==0):print( i, end=" " )

    # Epoch Loss & Accuracy
    print(dev_loss)
    print(dev_acc)
    total_loss = sum(dev_loss)/len(dev_loss);  
    total_acc = sum(dev_acc)/len(dev_acc);

    total_vol_loss = sum(dev_vol_loss)/len(dev_vol_loss); total_vol_acc = sum(dev_vol_acc)/len(dev_vol_acc);  
    total_ch_loss  = sum(dev_ch_loss )/len(dev_ch_loss ); total_ch_acc  = sum(dev_ch_acc )/len(dev_ch_acc );
    total_sub_loss = sum(dev_sub_loss)/len(dev_sub_loss); total_sub_acc = sum(dev_sub_acc)/len(dev_sub_acc);
    
    print('\n')
    print(f"End of {MODE} Process:")
    print(f"Total Loss: {total_loss} [ {total_vol_loss}, {total_ch_loss}, {total_sub_loss}] ")
    print(f"Total Accr: {total_acc } [ {total_vol_acc }, {total_ch_acc }, {total_sub_acc }]\n")    

    print(f"{MODE} Loss Epoch: {[ np.round(rloss , 3) for rloss in dev_loss]}")
    print(f"{MODE} Accuracy Epoch: {dev_acc}" )

    return total_loss, total_vol_loss, total_ch_loss, total_sub_loss, total_acc, total_vol_acc, total_ch_acc, total_sub_acc

In [ ]:
mFILE = "models/"
rFILE = "mResults/"

patience = MAX_PATIENCE

today = str(date.today())
Name = type(model).__name__ + "--" + today + "-"

# Save values for Training Set -- and --  Validation Set
Train_Loss=[]; Train_Acc = [];            Valid_Loss=[]; Valid_Acc = [];
Train_Vol_Loss = [];  Train_Vol_Acc = []; Valid_Vol_Loss = [];  Valid_Vol_Acc = [];   
Train_Ch_Loss  = [];  Train_Ch_Acc  = []; Valid_Ch_Loss  = [];  Valid_Ch_Acc  = []; 
Train_Sub_Loss = [];  Train_Sub_Acc = []; Valid_Sub_Loss = [];  Valid_Sub_Acc = []; 


tr_check_point = int(training_set.len/10) 
dev_check_point = int(develop_set.len/10)
test_check_point = int(testing_set.len/10)

print(tr_check_point, dev_check_point, test_check_point)

In [ ]:
Name

In [ ]:
EPOCHS = 3

In [ ]:
p1=torch.ones(TRAIN_BATCH_SIZE)*3 #

tr_check_point = 150
tr_check_point = 150

log_f = open(rFILE + Name + "-" + "system_log.txt", 'w', encoding='utf-8');  
log_f.write(f"Model: {Name}\n\n")


for epoch in range(EPOCHS ):

    print(f" ======================    Epoch: {epoch}    ======================\n")
    log_f.write(f" ================    Epoch: {epoch}    ======================\n"); 

    # Training Part
    tic = time.time()
    print("Training:\n")
    Tr_Loss,Tr_Vol_Loss,Tr_Ch_Loss,Tr_Sub_Loss,Tr_Acc,Tr_Vol_Acc,Tr_Ch_Acc,Tr_Sub_Acc \
                                                  = train(epoch, check_point = tr_check_point)
    log_f.write("Training:\n")
    log_f.write(f"\tLoss: {Tr_Loss}: [ {Tr_Vol_Loss}, {Tr_Ch_Loss}, {Tr_Sub_Loss} ]\n")
    log_f.write(f"\tAccu: {Tr_Acc }: [ {Tr_Vol_Acc }, {Tr_Ch_Acc }, {Tr_Sub_Acc } ]\n\n")
    

    # Validation Part
    print("\n\nValidation:\n")
    Dev_loss, Dev_vol_loss, Dev_ch_loss, Dev_sub_loss, Dev_acc, Dev_vol_acc, Dev_ch_acc, Dev_sub_acc \
                                                  = valid(model, testing_loader, )
    log_f.write(f"Validation:\n");
    log_f.write(f"\tLoss: {Dev_loss} [ {Dev_vol_loss}, {Dev_ch_loss}, {Dev_sub_loss}]\n")
    log_f.write(f"\tAccr: {Dev_acc } [ {Dev_vol_acc }, {Dev_ch_acc }, {Dev_sub_acc }]\n")

    # Save Epoch's Results
    toc = time.time()
    print(f"Epoch Duration: { (toc -tic)/60 } min\n")
    log_f.write(f"Epoch Duration: { (toc -tic)/60 } min\n")

    # Save Results from:  Training  &  Development Set
    Train_Loss.append(Tr_Loss); Train_Acc.append(Tr_Acc);
    Train_Vol_Loss.append(Tr_Vol_Loss);  Train_Vol_Acc.append(Tr_Vol_Acc);  
    Train_Ch_Loss.append( Tr_Ch_Loss)  ; Train_Ch_Acc.append( Tr_Ch_Acc); 
    Train_Sub_Loss.append(Tr_Sub_Loss);  Train_Sub_Acc.append(Tr_Sub_Acc); 

    Valid_Loss.append(Dev_loss); Valid_Acc.append(Dev_acc);
    Valid_Vol_Loss.append(Dev_vol_loss);  Valid_Vol_Acc.append(Dev_vol_acc);  
    Valid_Ch_Loss.append( Dev_ch_loss)  ; Valid_Ch_Acc.append( Dev_ch_acc); 
    Valid_Sub_Loss.append(Dev_sub_loss);  Valid_Sub_Acc.append(Dev_sub_acc);


    # Save Best Model Based on Total Loss
    if (Valid_Loss[-1] <= min(Valid_Loss)):

      patience = MAX_PATIENCE;
      mName = mFILE + Name+"-model-"+str(epoch)+".pt"
      torch.save(model.state_dict(), mName)
      log_f.write(f"Save Model: {mName} on Epoch {epoch} \n\n")
      print()
      print( f"Saved Model: {mName} of Epoch: {epoch}.\n")
      print(f"Patience: {patience}")

    else:
      patience = patience - 1
    if (patience == 0):break
    print('\n\n')


log_f.close()
print("End of Training Process")

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

x_axis = [i+1 for i in range(EPOCHS)]

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(211)
ax.plot( x_axis, Train_Acc, marker="*", color="black", markersize=12)
ax.plot( x_axis, Valid_Acc, marker="*", color="black", markersize=12)
ax.plot( x_axis, Train_Acc, label='Train Accuracy' , color='red', linewidth=2.5) ;
ax.plot( x_axis, Valid_Acc, label='Valid Accuracy', color='purple', linewidth=2.5)
plt.title(f"{Name}--Accuracy"); ax.legend(loc="lower right")

ax = fig.add_subplot(212)
ax.plot( x_axis, Train_Loss, marker="*", color="navy", markersize=12)
ax.plot( x_axis, Valid_Loss, marker="*", color="navy", markersize=12)
ax.plot( x_axis, Train_Loss, label='Train Loss', color='red', linewidth=2.5)
ax.plot( x_axis, Valid_Loss, label='Valid Loss', color='purple', linewidth=2.5)
plt.title(f"{Name}--Loss");  ax.legend()

plt.savefig("mResults/"+Name+"-Acc\Loss-Plot.png")
plt.show()

---
# **`TESTING SECTION`**

---

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
FILE = "mResults/" # log-file Position
#tName = "models/Last3_BERT--2021-05-03--model-2.pt" # Saving Position
tName = "models/SumLast_BERT--2021-05-05--model-2.pt" # Saving Position
tName = "models/SumLast4_BERT--2021-05-06--model-2.pt"

In [ ]:
#Model = SimpleBERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub,)
#Model = Last3_BERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub,)
#Model = Mask_SimpleBERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub, Chapter_matrix= Ch_matrix, Subject_matrix=Sub_matrix)
Model = SumLast4_BERT(bert_model=BERT_MODEL, vol_num=Unique_Vol , ch_num= Unique_Ch, sub_num= Unique_Sub)

Model.to(device)
Model.load_state_dict(torch.load( tName ))
Model.eval()

In [ ]:
log_f = open( FILE + Name +"-"+'system_log.txt', 'a', encoding='utf-8'); 
log_f.write(f"\n\n\n"); log_f.flush()


Test_loss, Test_vol_loss, Test_ch_loss, Test_sub_loss, Test_acc, Test_vol_acc, Test_ch_acc, Test_sub_acc \
                                        = valid(model, testing_loader, test_check_point, TEST_BATCH_SIZE, "Test")

# Write Results on log_f File

log_f.write(f" ================    TESTING    ======================\n");
log_f.write(f"Loss: {Test_loss} [ {Test_vol_loss}, {Test_ch_loss}, {Test_sub_loss}]\n")
log_f.write(f"Accr: {Test_acc } [ {Test_vol_acc }, {Test_ch_acc }, {Test_sub_acc }]\n\n\n")
log_f.close()

---
**Accuracy--Macro_Avg--Weighted_Avg**

---

In [ ]:
Model = model

In [ ]:
true_vol_output = []; pred_vol_output = []
true_ch_output  = []; pred_ch_output  = []
true_sub_output = []; pred_sub_output = []

for i ,data in enumerate(develop_loader, 0):
  ids = data['ids'].to(device, dtype = torch.long)
  mask = data['mask'].to(device, dtype = torch.long)

  # Target values for Volume, Chapter and Subject Levels
  target_vol = data['target_vol'].to(device, dtype = torch.long)
  target_ch = data['target_ch'].to(device, dtype = torch.long)
  target_sub = data['target_sub'].to(device, dtype = torch.long)

  if (target_vol.shape[0]!=TEST_BATCH_SIZE): break

  true_vol_output.extend(target_vol.tolist())
  true_ch_output.extend (target_ch.tolist() )
  true_sub_output.extend(target_sub.tolist())

  vol, ch, sub = Model(ids, mask)  
  #print(vol)

  vol_big_val, vol_big_idx = torch.max(vol.data, dim=1)
  #print(vol_big_idx.tolist(), target_vol)

  ch_big_val, ch_big_idx  = torch.max(ch.data, dim=1)
  #print(ch_big_idx.tolist(), target_ch)

  sub_big_val, sub_big_idx = torch.max(sub.data, dim=1)
  #print(sub_big_idx.tolist(), target_sub )

  pred_vol_output.extend(vol_big_idx.tolist())
  pred_ch_output.extend (ch_big_idx.tolist() )
  pred_sub_output.extend(sub_big_idx.tolist())

  vol_acc, inst1 = calculate_accu(vol_big_idx, target_vol) 
  ch_acc,  inst2 = calculate_accu(ch_big_idx, target_ch)   
  sub_acc, inst3 = calculate_accu(sub_big_idx, target_sub) 

  insts= torch.tensor([ inst1.tolist(), inst2.tolist(), inst3.tolist()])
  #print(insts)

  p1=torch.ones(4)*3
  total_acc = (insts.sum(dim=0)==p1).sum().item()/4
  #print(f"Acc: {total_acc} [ {vol_acc}, {ch_acc}, {sub_acc}]" )


In [ ]:
print(len(true_ch_output), len(pred_ch_output),"\n")

print(true_vol_output[:50])
print(pred_vol_output[:50])
print(len(list(np.unique(np.array(pred_vol_output)))), list(np.unique(np.array(pred_vol_output))))
print(len(list(np.unique(np.array(true_vol_output)))), list(np.unique(np.array(true_vol_output))))

print() 
print( true_ch_output [:50])
print( pred_ch_output [:50])
print(len(list(np.unique(np.array(pred_ch_output)))), list(np.unique(np.array(pred_ch_output))))
print(len(list(np.unique(np.array(true_ch_output)))), list(np.unique(np.array(true_ch_output))))

print() 
print(true_sub_output[:50])
print(pred_sub_output[:50])
print(len(list(np.unique(np.array(pred_sub_output)))), list(np.unique(np.array(pred_sub_output))))
print(len(list(np.unique(np.array(true_sub_output)))), list(np.unique(np.array(true_sub_output))))

In [ ]:
log_f = open( FILE + Name +"-"+'system_log.txt', 'a', encoding='utf-8'); 

# VOLUMES:  Accuracy-Macro_avg-Weighted-Avg
vol_cr_output = classification_report(  y_true=true_vol_output,  y_pred=pred_vol_output, output_dict=True,
                              target_names=[get_key(vol_label_dict, i)[0] for i in range(len(vol_label_dict))],
                              labels=[i for i in range(len(vol_label_dict))], )

log_f.write("VOLUMES:  Accuracy---Macro_avg---Weighted-Avg\n")
log_f.write(str(vol_cr_output["accuracy"])    ); log_f.write("\n")
log_f.write(str(vol_cr_output["macro avg"])   ); log_f.write("\n")
log_f.write(str(vol_cr_output["weighted avg"])); log_f.write("\n\n\n")


# CHAPTERS:  Accuracy-Macro_avg-Weighted-Avg
ch_cr_output = classification_report(  y_true=true_ch_output,  y_pred=pred_ch_output, output_dict=True,)
                              #target_names=[get_key(ch_label_dict, i)[0] for i in range(len(ch_label_dict))],
                              #labels=[i for i in range(len(ch_label_dict))], )

log_f.write("CHAPTERS:  Accuracy---Macro_avg---Weighted-Avg\n")
log_f.write(str(ch_cr_output["accuracy"])    ); log_f.write("\n")
log_f.write(str(ch_cr_output["macro avg"])   ); log_f.write("\n")
log_f.write(str(ch_cr_output["weighted avg"])); log_f.write("\n\n\n")


# SUBJECTS:  Accuracy-Macro_avg-Weighted-Avg
sub_cr_output = classification_report(  y_true=true_sub_output,  y_pred=pred_sub_output, output_dict=True,
                              target_names=[get_key(sub_label_dict, i)[0] for i in range(len(sub_label_dict))],
                              labels=[i for i in range(len(sub_label_dict))], )

log_f.write("SUBJECTS:  Accuracy---Macro_avg---Weighted-Avg\n")
log_f.write(str(sub_cr_output["accuracy"])    ); log_f.write("\n")
log_f.write(str(sub_cr_output["macro avg"])   ); log_f.write("\n")
log_f.write(str(sub_cr_output["weighted avg"])); log_f.write("\n\n\n")

log_f.close()

In [ ]:
print(classification_report(  y_true=true_vol_output,  y_pred=pred_vol_output,
                              target_names=[get_key(vol_label_dict, i)[0] for i in range(len(vol_label_dict))],
                              labels=[i for i in range(len(vol_label_dict))],
                                #labels=[e for e in inv_class_dict.keys() if e != 15],
                                #target_names=[e for e in class_dict.keys() if e != 'X'],
                                output_dict=False))

In [ ]:
print(classification_report(  y_true=true_ch_output,  y_pred=pred_ch_output,
                              target_names=[get_key(ch_label_dict, i)[0] for i in range(len(ch_label_dict))],
                              labels=[i for i in range(len(ch_label_dict))],
                                #labels=[e for e in inv_class_dict.keys() if e != 15],
                                #target_names=[e for e in class_dict.keys() if e != 'X'],
                                output_dict=False))

In [ ]:
print(classification_report(  y_true=true_sub_output,  y_pred=pred_sub_output,
                              target_names=[get_key(sub_label_dict, i)[0] for i in range(len(sub_label_dict))],
                              labels=[i for i in range(len(sub_label_dict))],
                                #labels=[e for e in inv_class_dict.keys() if e != 15],
                                #target_names=[e for e in class_dict.keys() if e != 'X'],
                                output_dict=False))